<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.1.1 
# *Data Wrangling and Munging with Pandas*

## Part 1: Wrangling Data

The term "data wrangling" is analogous to capturing wild horses and getting them into a fenced area; the horses are data and the fencing is your computer. The more common data wrangling tasks include:

- reading flat files
- reading Excel files
- downloading from web pages
  - csv
  - html
  - json

In [1]:
import numpy as np
import pandas as pd

*It is good practice to display the library version numbers for future reference:*

In [2]:
print('Numpy: ', np.__version__)
print('Pandas: ', pd.__version__)

Numpy:  1.20.1
Pandas:  1.2.4


### CSV Files

Below are three attempts to load the file "bikeshare.csv" into a DataFrame named `bikes`. Why are they wrong?

In [3]:
# wrong:
bikes = pd.read_table('dat/bikeshare.csv', header = None)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('dat/bikeshare.csv', header = 1)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('dat/bikeshare.csv', header = 0)
print(bikes.head())

FileNotFoundError: [Errno 2] No such file or directory: 'dat/bikeshare.csv'

?:
ANSWER: Case 1 treats headings as just another data row. Case 2 treats the 1st data row as the column header. Case 3 gets the header right (row 0), but reads each row as a single column (Nb. the other two make that same mistake). 

Load the file "bikeshare.csv" into a DataFrame named `bikes`, and confirm that it was loaded properly:

In [4]:
#ANSWER:
bikes = pd.read_csv('D:\IOD Data/bikeshare.csv')
bikes = pd.DataFrame(bikes)
print(bikes)

                  datetime  season  holiday  workingday  weather   temp  \
0      2011-01-01 00:00:00       1        0           0        1   9.84   
1      2011-01-01 01:00:00       1        0           0        1   9.02   
2      2011-01-01 02:00:00       1        0           0        1   9.02   
3      2011-01-01 03:00:00       1        0           0        1   9.84   
4      2011-01-01 04:00:00       1        0           0        1   9.84   
...                    ...     ...      ...         ...      ...    ...   
10881  2012-12-19 19:00:00       4        0           1        1  15.58   
10882  2012-12-19 20:00:00       4        0           1        1  14.76   
10883  2012-12-19 21:00:00       4        0           1        1  13.94   
10884  2012-12-19 22:00:00       4        0           1        1  13.94   
10885  2012-12-19 23:00:00       4        0           1        1  13.12   

        atemp  humidity  windspeed  casual  registered  count  
0      14.395        81     0.0000 

Note that we could have used `read.csv()` above. When is `read_table()` necessary?

?:
ANSWER: When `sep` is not the comma character, or we need fine control that `read.csv()` does not provide.

Flat files can be full of surprises. Here are some issues to watch out for:

- separator character is something other than the comma
  - ";", "|", and tab are popular
- newline character is something other than what the O/S expects 
  - Tip: Don't hard-code the character codes for carriage returns, linefeeds, etc. Use Python's built-in representation instead (e.g. Python translates "\n" to the newline character and "\t" to the tab character on any O/S).
- truncated lines
  - if there are empty fields at the end of a line it is possible that their separators will be missing, resulting in a "jagged" file
- embedded commas or quotes
  - a free-text field containing embedded commas may split into separate fields on input
  - a free-text field containing embedded quotes may not parse correctly
- unescaped characters
  - the "\" character indicates a control code to Python, which will break the I/O
    - e.g. the substring "\u0123" will be interpreted as Unicode(0123) -- which may not be what the file creator intended
  - these may need to be fixed by loading whole strings and then parsing into a new data frame
  
Tip: Most issues can be delth with by correctly specifying the parameters of the function you use to load the file. Read the doco before reading the data!

### Reading Excel Files

In [5]:
from pandas import ExcelFile  # Nb. Need to install xlrd from conda (it does not automatically install with pandas)

In [6]:
df = pd.read_excel('dat/Iris.xls', sheet_name = 'Iris')
df

FileNotFoundError: [Errno 2] No such file or directory: 'dat/Iris.xls'

So, this file appears to have an embedded table of aggregates on the same sheet as the raw data (a naughty but common practice amongst analysts).

It is usually better to load data correctly than to meddle with the source file or load it 'warts and all' and then try to parse it in code. The Pandas functions for reading files have parameters that provide the control we need. For ecxample, we could make multiple calls to `read_excel()`, using combinations of the `header`, `usecols`, `skiprows`, `nrows`, and `skipfooter` parameters to load one table at a time from a spreadsheet with multiple tables.

Load the above file without the unwanted columns:

In [7]:
#ANSWER
df = pd.read_excel('dat/Iris.xls', sheet_name = 'Iris', header = 0, usecols =","

SyntaxError: unexpected EOF while parsing (<ipython-input-7-99ea6a79699d>, line 2)

### Importing Data Directly from the Web

We usually want to store a local copy of a data file that we download from the Web, but when data retention is not a priority it is convenient to download the data directly into our running Python environment.

#### Importing Text Files from the Web

The web is the 'wild west' of data formats. However, we can usually expect good behaviour from files that are automatically generated by a service, such as the earthquake report:

In [8]:
df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_hour.csv')
df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2021-07-17T03:57:54.868Z,31.354401,-103.077942,3.457935,3.1,ml,42.0,52,0.018414,0.20,...,2021-07-17T04:17:06.557Z,"12 km N of Coyanosa, Texas",earthquake,1.239859,2.723292,0.200,32.0,reviewed,tx,tx
1,2021-07-17T03:23:47.354Z,-20.623800,-178.380400,559.130000,5.5,mww,NaN,35,4.423000,0.92,...,2021-07-17T04:13:28.040Z,Fiji region,earthquake,8.000000,6.900000,0.098,10.0,reviewed,us,us
2,2021-07-17T03:19:44.360Z,-23.459900,-67.001100,234.870000,4.2,ml,NaN,117,1.195000,0.45,...,2021-07-17T03:46:24.040Z,"109 km NW of San Antonio de los Cobres, Argentina",earthquake,12.100000,13.500000,NaN,NaN,reviewed,us,guc


#### Importing HTML Files from the Web

Working with unstructured HTML files relies heavily on library functions. This one, however, is well-structured:

In [9]:
url = 'https://www.fdic.gov/resources/resolutions/bank-failures/failed-bank-list/'

df = pd.read_html(url)
df

[                         Bank NameBank           CityCity StateSt  CertCert  \
 0                    Almena State Bank             Almena      KS     15426   
 1           First City Bank of Florida  Fort Walton Beach      FL     16748   
 2                 The First State Bank      Barboursville      WV     14361   
 3                   Ericson State Bank            Ericson      NE     18265   
 4     City National Bank of New Jersey             Newark      NJ     21111   
 ..                                 ...                ...     ...       ...   
 558                 Superior Bank, FSB           Hinsdale      IL     32646   
 559                Malta National Bank              Malta      OH      6629   
 560    First Alliance Bank & Trust Co.         Manchester      NH     34264   
 561  National State Bank of Metropolis         Metropolis      IL      3815   
 562                   Bank of Honolulu           Honolulu      HI     21029   
 
                  Acquiring Institutio

#### Importing XML Files from the Web

XML files are semi-structured, but you're at the mercy of the file creator. If every record has the same format it will be much easier, but practical applications often require a lot of custom code. Here is an example that includes a nice parser class: http://www.austintaylor.io/lxml/python/pandas/xml/dataframe/2016/07/08/convert-xml-to-pandas-dataframe/

#### Importing JSON Files from the Web

Like XML, JSON files are semi-structured and may require work to capture the schema into a dataframe. Here is a simple example: 

In [10]:
url = 'https://raw.githubusercontent.com/chrisalbon/simulated_datasets/master/data.json'

# Load the first sheet of the JSON file into a data frame
df = pd.read_json(url, orient = 'columns')
df.head()

,integer,datetime,category
0,5,2015-01-01 00:00:00,0
1,5,2015-01-01 00:00:01,0
2,9,2015-01-01 00:00:02,0
3,6,2015-01-01 00:00:03,0
4,6,2015-01-01 00:00:04,0


## Part 2: Data Munging

Data munging is manipulating data to get it into a form that we can start running analyses on (which usually means getting the data into a DataFrame). Before we get to this stage, we may need to remove headers or footers, transpose columns to rows, split wide data tables into long ones, and so on. (Nb. Excel files can be particularly troublesome, because users can format their data in mixed, complex shapes.) Essentially, we need to follow Hadley Wickham's guidelines for tidy datasets (http://vita.had.co.nz/papers/tidy-data.html):

The end goal of the cleaning data process:

- each variable should be in one column
- each observation should comprise one row
- each type of observational unit should form one table
- include key columns for linking multiple tables
- the top row contains (sensible) variable names
- in general, save data as one file per table


### Dataset Morphology

Once we have our dataset in a DataFrame (or Series, if our data is only 1-dimensional), we can start examining its size and content.

How many rows and columns are in `bikes`?

In [11]:
#ANSWER
10886 rows x 12 columns

SyntaxError: invalid syntax (<ipython-input-11-b70534148bee>, line 2)

What are the column names in `bikes`?

In [12]:
#ANSWER
col = bikes.columns
print(col)

Index(['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed', 'casual', 'registered', 'count'],
      dtype='object')


What are the data types of these columns?

In [76]:
#ANSWER
bikes.dtypes

datetime         object
season            int64
holiday           int64
workingday        int64
weather           int64
temp            float64
atemp           float64
humidity          int64
windspeed       float64
casual            int64
registered        int64
count             int64
atemp_level    category
dtype: object

What is the (row) index for this DataFrame?

In [14]:
#ANSWER
df = pd.DataFrame(bikes)
print(df.dtypes)

datetime       object
season          int64
holiday         int64
workingday      int64
weather         int64
temp          float64
atemp         float64
humidity        int64
windspeed     float64
casual          int64
registered      int64
count           int64
dtype: object


https://www.dataquest.io/blog/python-json-tutorial/

## Slicing and Dicing

It is often preferable to refer to DataFrame columns by name, but there is more than one way to do this. 
Do `bikes['season']` and `bikes[['season']]` give the same object? Demonstrate:

In [15]:
#ANSWER
print(bikes['season'])
print(bikes[['season']])

0        1
1        1
2        1
3        1
4        1
        ..
10881    4
10882    4
10883    4
10884    4
10885    4
Name: season, Length: 10886, dtype: int64
       season
0           1
1           1
2           1
3           1
4           1
...       ...
10881       4
10882       4
10883       4
10884       4
10885       4

[10886 rows x 1 columns]


How would we use object notation to show the first 4 rows of `atemp`?

In [16]:
#ANSWER
print(df.loc[0:3, 'atemp'])

0    14.395
1    13.635
2    13.635
3    14.395
Name: atemp, dtype: float64


Algorithms that loop over multiple columns often access DataFrame columns by index. However, none of the following work (try them out by uncommenting / removing the "#E: " ): 

In [17]:
bikes[[0]]
bikes[0]
bikes[0,0]
bikes[[0,0]]

KeyError: "None of [Int64Index([0], dtype='int64')] are in the [columns]"

What is the correct way to access the 1st row of the DataFrame by its index?

In [18]:
#ANSWER
df[:1]
#df.head(1) also works


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16


What is the correct way to access the 2nd column of the DataFrame by its index?

In [19]:
#ANSWER
df.iloc[:, 1]

0        1
1        1
2        1
3        1
4        1
        ..
10881    4
10882    4
10883    4
10884    4
10885    4
Name: season, Length: 10886, dtype: int64

## Handling Missing Values

What is the Pandas `isnull` function for? 

?
ANSWER:

We can apply `isnull` to the `bikes` DataFrame to show the result for every element:

In [20]:
bikes.isnull().head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False


However, we usually start at a higher level. How many nulls are in `bikes` altogether?

In [21]:
#ANSWER
print(bikes.isnull())
len(bikes.isnull())

       datetime  season  holiday  workingday  weather   temp  atemp  humidity  \
0         False   False    False       False    False  False  False     False   
1         False   False    False       False    False  False  False     False   
2         False   False    False       False    False  False  False     False   
3         False   False    False       False    False  False  False     False   
4         False   False    False       False    False  False  False     False   
...         ...     ...      ...         ...      ...    ...    ...       ...   
10881     False   False    False       False    False  False  False     False   
10882     False   False    False       False    False  False  False     False   
10883     False   False    False       False    False  False  False     False   
10884     False   False    False       False    False  False  False     False   
10885     False   False    False       False    False  False  False     False   

       windspeed  casual  r

10886

If this result were nonzero we would next want to find out which columns contained nulls. How can this be done in one line of code?

In [22]:
#ANSWER
print(bikes.isnull().sum())

datetime      0
season        0
holiday       0
workingday    0
weather       0
temp          0
atemp         0
humidity      0
windspeed     0
casual        0
registered    0
count         0
dtype: int64


What is the Numpy object `nan` used for? (Write a descriptive answer.)

?
ANSWER: Marking a data point as invalid.

Write (and verify) a function that performs scalar division with built-in handling of the edge case (i.e. return a value instead of just trapping the error):

In [23]:
#ANSWER
if np.nan return nill
nill = print("Null")

SyntaxError: invalid syntax (<ipython-input-23-bbcf30c581b2>, line 2)

Apply the Pandas `isna` function to the following data objects:

In [24]:
x1 = 2.3
y1 = np.nan
print(x, y)

NameError: name 'x' is not defined

In [25]:
#ANSWER
x1 = 2.3
print(pd.isna(x1))
y1 = np.nan
print(pd.isna(y1))

False
True


In [26]:
array = np.array([[1, np.nan, 3], [4, 5, np.nan]])
print(array)

[[ 1. nan  3.]
 [ 4.  5. nan]]


In [27]:
#ANSWER
array = np.array([[1, np.nan, 3], [4, 5, np.nan]])
print(pd.isna(array))

[[False  True False]
 [False False  True]]


How is the pandas I/O parameter `na_values` used?

? ANSWER: represent misssing values

## Data Profiling

### Counts

When there are categorical variables in a dataset we will want to know how many possible values there are in each column. (Nb. If the dataset is a sample of a larger one, our sample may not capture all possible values of every categorical.)

How many (different) seasons are in `bikes`?

In [28]:
#ANSWER
bikes['season'].nunique()

4

### Ranges

Print the range of the `instant`, `dteday`, and `windspeed` columns: 

In [29]:
#ANSWER
print(df.iloc[:0, 1:4])
print(bikes)

Empty DataFrame
Columns: [season, holiday, workingday]
Index: []
                  datetime  season  holiday  workingday  weather   temp  \
0      2011-01-01 00:00:00       1        0           0        1   9.84   
1      2011-01-01 01:00:00       1        0           0        1   9.02   
2      2011-01-01 02:00:00       1        0           0        1   9.02   
3      2011-01-01 03:00:00       1        0           0        1   9.84   
4      2011-01-01 04:00:00       1        0           0        1   9.84   
...                    ...     ...      ...         ...      ...    ...   
10881  2012-12-19 19:00:00       4        0           1        1  15.58   
10882  2012-12-19 20:00:00       4        0           1        1  14.76   
10883  2012-12-19 21:00:00       4        0           1        1  13.94   
10884  2012-12-19 22:00:00       4        0           1        1  13.94   
10885  2012-12-19 23:00:00       4        0           1        1  13.12   

        atemp  humidity  windspeed

Compute and print the overall minimum and maximum of the numeric data columns:

In [30]:
bike_col = df['temp']
bike_min = bike_col.min()
bike_max = bike_col.max()
print(bike_min, bike_max)

0.82 41.0


### Quantiles

Pandas makes computing quantiles easy. This is how to get the median of a Series:

In [31]:
bikes['atemp'].quantile(0.5)

24.24

Of course, the `quantiles` method can take a tuple as its argument. Compute the 10th, 25th, 50th, 75th, and 90th percentiles in one line of code: 

In [32]:
#ANSWER
print(bikes['atemp'].quantile([.10,.25,.50,.75,.90]))

0.10    12.120
0.25    16.665
0.50    24.240
0.75    31.060
0.90    34.090
Name: atemp, dtype: float64


### Cuts

Sometimes we want to split the sample not by the quantiles of the distribution but by the range of the data. Let's take a closer look at `atemp`:

In [33]:
type(bikes['atemp'])

pandas.core.series.Series

In [34]:
bikes.sample(5)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
3245,2011-08-05 03:00:00,3,0,1,1,26.24,29.545,78,6.0032,0,6,6
3408,2011-08-11 22:00:00,3,0,1,1,28.70,32.575,48,0.0000,34,123,157
2332,2011-06-05 02:00:00,2,0,0,2,26.24,31.060,61,11.0014,16,48,64
10148,2012-11-08 06:00:00,4,0,1,1,12.30,14.395,39,15.0013,2,122,124
3353,2011-08-09 15:00:00,3,0,1,1,35.26,38.635,39,19.0012,58,106,164


Suppose we decide to sort these values into 4 bins of equal width, but we want to apply the resulting groups to the entire DataFrame. Basically, we need to add a row label that indcates which bin each sample belongs in. Let's call this label "atemp_level", and use the `cut` method to populate it:

In [35]:
atemp_level = pd.cut(bikes['atemp'], bins = 4)    

What is `atemp_level`?

In [36]:
#ANSWER
catergorized atemp into 4 bins

SyntaxError: invalid syntax (<ipython-input-36-9bae83ecf98b>, line 2)

Here is a random sample of `atemp_level`:

In [37]:
atemp_level.sample(5)          

8727    (34.281, 45.455]
5191    (11.934, 23.108]
912     (11.934, 23.108]
2837    (23.108, 34.281]
4722    (11.934, 23.108]
Name: atemp, dtype: category
Categories (4, interval[float64]): [(0.715, 11.934] < (11.934, 23.108] < (23.108, 34.281] < (34.281, 45.455]]

So, by default, `cut` produces labels that indicate the bin boundaries for each element in the series it was applied to. Usually, we will specify labels that are appropriate to the discretisation we are applying:

In [38]:
atemp_level = pd.cut(bikes['atemp'], bins = 4, labels = ["cool", "mild", "warm", "hot"])
atemp_level.sample(5)          

823     warm
8678     hot
456     cool
8085    warm
1469    mild
Name: atemp, dtype: category
Categories (4, object): ['cool' < 'mild' < 'warm' < 'hot']

Incorporate the new `atemp_level` column into the `bikes` DataFrame and use it to count the number of "mild" `atemp` entries in `season` 2:

In [43]:
#ANSWER
df['atemp_level'] = atemp_level
print(df['season'] == 2 & df['atemp_level'] == "mild")

TypeError: unsupported operand type(s) for &: 'int' and 'Categorical'

*Nb. The `atemp_level` variable we created is what the R language calls a "factor". Pandas has introduced a new data type called "category" that is similar to R's factors.*

# Synthetic Data

Sometimes we may want to generate test data, or we may need to initalise a series, matrix, or data frame for input to an algorithm. Numpy has several methods we can use for this.

Execute the following, then check the shape and content of each variable:

In [50]:
# Creating arrays with initial values
a = np.zeros((3))
b = np.ones((1,3))
c = np.random.randint(1,10,(2,3,4))   # randint(low, high, size)
d = np.arange(4)
e = np.array( [[1,2,3,4], [5,6,7,8]] )
print(np.shape(a))
print(np.shape(b))
print(np.shape(c))
print(np.shape(d))
print(np.shape(e))


(3,)
(1, 3)
(2, 3, 4)
(4,)
(2, 4)


In [41]:
# Cleaning Data

## Load Data

Load rock.csv and clean the dataset.


In [55]:
rock = pd.read_csv('D:\IOD Data/rock.csv')

KeyError: (0, 1)

## Check Column Names

Check column names and clean.

In [56]:
print(rock.columns.values)

['Song Clean' 'ARTIST CLEAN' 'Release Year' 'COMBINED' 'First?' 'Year?'
 'PlayCount' 'F*G']


## Replace Null Values With 0

Check 'release' column whether this column have any null value or not. Replace null value with 0.

In [59]:
print(rock['Release Year'].fillna(0))

0       1982
1          0
2       1981
3       1980
4       1975
        ... 
2224       0
2225    1981
2226    1975
2227    1983
2228    1973
Name: Release Year, Length: 2229, dtype: object


## Check Datatypes of Dataset

Check datatypes of the dataset. Is there any column which should be int instead of object? Fix the column. 

In [75]:
df_rock = pd.DataFrame(rock)
df_rock.dtypes
df_rock = df_rock.astype({"F*G": int})

## Check Min, Max of Each Column

Is there any illogical value in any column? How can we fix that?

## Write Some Functions

## Write a function that will take a row of a DataFrame and print out the song, artist, and whether or not the release date is < 1970

## Write a function that converts cells in a DataFrame to float and otherwise replaces them with np.nan

## Apply these functions to your dataset

## Describe the new float-only DataFrame.

>




---



---



> > > > > > > > > © 2021 Institute of Data


---



---



